In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import matplotlib.pyplot as plt
import numpy as np
import scipy
import json
import pandas as pd
import seaborn as sns
import os
import admix
import scipy.stats

In [ ]:
def estimate_rg(est, est_var):
    """
    Estimate the ratio of genetic correlation.
    est: (3, ) numpy array
    est_var (3, 3) variance-covariance matrix
    """
    x, y = est[0], est[1]
    rg = y / x
    # grad = [-y / x^2, 1 / x]
    grad = np.array([-y / (x ** 2), 1 / x])

    def quad_form(x, A):
        return np.dot(np.dot(x.T, A), x)

    return rg, quad_form(grad, est_var[0:2, 0:2])

In [ ]:
df_plot = []

for hsq in [0.1, 0.25, 0.5]:
    for pcausal in [0.00001, 0.0001, 0.001, 0.01]:
        print(f"Loading hsq={hsq}, pcausal={pcausal}")
        for cor in [0.9, 0.95, 1.0]:
            for hermodel in ["mafukb"]:
                for sim_i in range(0, 500):
                    f_name = (
                        f"out/gcta-estimate/hsq-{hsq}-pcausal-{np.format_float_positional(pcausal)}-cor-{cor}"
                        + f"-hermodel-{hermodel}.sim_{sim_i}"
                    )
                    if not os.path.exists(f_name + ".hsq"):
                        continue
                    try:
                        est = admix.tools.gcta.read_reml(f_name)
                        rg, rg_var = estimate_rg(
                            est["est"].Variance.values, est["varcov"].values
                        )
                        rg_stderr = np.sqrt(rg_var)

                        # results from likelihood ratio test
                        reduced_est = admix.tools.gcta.read_reml(f_name + ".reduced")
                        pval_lrt = scipy.stats.chi2.sf(
                            2 * (est["loglik"] - reduced_est["loglik"]), 1
                        )
                        pval_delta = scipy.stats.norm.cdf((rg - 1) / rg_stderr)
                        df_plot.append(
                            est["est"].Variance.values.tolist()
                            + [
                                hsq,
                                cor,
                                pcausal,
                                hermodel,
                                rg,
                                rg_stderr,
                                pval_lrt,
                                pval_delta,
                                sim_i,
                            ]
                        )
                    except ValueError:
                        print("Invalid value!")
df_plot = pd.DataFrame(
    df_plot,
    columns=[
        "estimated_var_g",
        "estimated_rho",
        "estimated_var_e",
        "hsq",
        "cor",
        "pcausal",
        "hermodel",
        "estimated_ratio",
        "estimated_ratio_stderr",
        "pval_lrt",
        "pval_delta",
        "sim_i",
    ],
)

In [ ]:
df_plot.to_csv("results/raw.csv", index=False)